***
# Nettoyage des données
***

In [3]:
# ====== Importer les library ====== #
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_columns', None)

In [4]:
# ====== Déterminer les path ====== #
import os
cwd = os.getcwd()
parent = os.path.dirname(cwd)
data_path = parent + '\\data\\'

In [5]:
# ====== Importer les df ====== #
comments = pd.read_csv(data_path + 'Comments.csv')
posts = pd.read_csv(data_path + 'Posts.csv')

comments.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,created_time,id,like_count,message,parent,postId
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,2023-06-02T16:06:59+0000,722477749883613_788168749338745,0,"Faux, ce ne sera jamais le temps pour lui car ...",NaN,192978590727638_722477749883613
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,2023-06-02T16:21:43+0000,722477749883613_1593974021087773,1,Mon dieu!! On nous prend sérieusement pour des...,NaN,192978590727638_722477749883613


In [6]:
test = pd.read_csv(data_path + 'Test.csv')
test.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,created_time,id,like_count,message,parent,postId
0,0.000548,0.007893,0.011800,0.000517,0.005419,0.006754,-1,2023-06-01T18:32:37+0000,721844289946959_831368741921536,0,Mon ex🤔?,NaN,192978590727638_721844289946959
1,0.125566,0.469236,0.501357,0.169603,0.012389,0.434752,-1,2023-06-01T18:51:50+0000,721844289946959_9565562033514339,0,"""Quel est ton nom ? Je te l' ordonne de le di...",NaN,192978590727638_721844289946959


In [146]:
# ====== Merge les df ====== #
dftest_merge = pd.merge(test, posts, left_on='postId', right_on='id', indicator=True, suffixes=('_comments', '_posts'))
dftest_merge.head(2)

# *** Un algo qui check si le titre est déjà dans le set de données? 

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,created_time_comments,id_comments,like_count,message_comments,parent,postId,attachments.data,created_time_posts,id_posts,mainTopic,message_posts,permalink_url,secondTopic,shares,title,_merge
0,0.000548,0.007893,0.011800,0.000517,0.005419,0.006754,-1,2023-06-01T18:32:37+0000,721844289946959_831368741921536,0,Mon ex🤔?,NaN,192978590727638_721844289946959,"[{""title"":""Un troupeau de voix enragées"",""unsh...",2023-06-01T18:31:33+0000,192978590727638_721844289946959,chroniques,CHRONIQUE / Entre 4 à 10% des Québécois entend...,https://www.facebook.com/553956616735728/posts...,patrick-duquette,0,Un troupeau de voix enragées,both
1,0.125566,0.469236,0.501357,0.169603,0.012389,0.434752,-1,2023-06-01T18:51:50+0000,721844289946959_9565562033514339,0,"""Quel est ton nom ? Je te l' ordonne de le di...",NaN,192978590727638_721844289946959,"[{""title"":""Un troupeau de voix enragées"",""unsh...",2023-06-01T18:31:33+0000,192978590727638_721844289946959,chroniques,CHRONIQUE / Entre 4 à 10% des Québécois entend...,https://www.facebook.com/553956616735728/posts...,patrick-duquette,0,Un troupeau de voix enragées,both


In [147]:
# ====== Merge les df ====== #
df_merge = pd.merge(comments, posts, left_on='postId', right_on='id', indicator=True, suffixes=('_comments', '_posts'))
df_merge.head(2)


# Analyse du merge 
unique_list = []
for value in df_merge['_merge'].unique():
    unique_list.append(value)

if (len(unique_list) == 1) & (unique_list[0] == 'both'):
    print('Tous les lignes merge')
else :
    print('Certaines lignes ne sont pas merge')


df_merge = df_merge.drop(columns='_merge')

Tous les lignes merge


In [148]:
# Attention au Nan
df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')

C:\Users\Charles_tour\AppData\Local\Temp\ipykernel_22784\284039997.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['created_time_comments', 'id_comments', 'message_comments', 'parent',
       'postId', 'attachments.data', 'created_time_posts', 'id_posts',
       'mainTopic', 'message_posts', 'permalink_url', 'secondTopic', 'title'],
      dtype='object')]

  df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')


***
### Compléter les valeurs manquantes

#### Solution
- Drop parent
- Drop missings title row 
***

In [149]:
df_merge =pd.read_hdf(cwd + '\\dataframe\\' + 'df_merge.h5')
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,created_time_comments,id_comments,like_count,message_comments,parent,postId,attachments.data,created_time_posts,id_posts,mainTopic,message_posts,permalink_url,secondTopic,shares,title
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,2023-06-02T16:06:59+0000,722477749883613_788168749338745,0,"Faux, ce ne sera jamais le temps pour lui car ...",NaN,192978590727638_722477749883613,"[{""title"":""Les incohérences de l’ingérence"",""u...",2023-06-02T16:03:24+0000,192978590727638_722477749883613,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,https://www.facebook.com/553956616735728/posts...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,2023-06-02T16:21:43+0000,722477749883613_1593974021087773,1,Mon dieu!! On nous prend sérieusement pour des...,NaN,192978590727638_722477749883613,"[{""title"":""Les incohérences de l’ingérence"",""u...",2023-06-02T16:03:24+0000,192978590727638_722477749883613,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,https://www.facebook.com/553956616735728/posts...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence


In [150]:
# ====== Visualisation des valeurs manquantes ====== #
print('Missing value')
print('*'*25)
print(f'Nombre de lignes : {len(df_merge)}')
for col in df_merge.columns.to_list():
    nb_na = df_merge[col].isna().sum()
    perc = round(df_merge[col].isna().sum()/len(df_merge)*100)
    print(f'{col.ljust(21)} : {str(nb_na).rjust(11)} ({perc}%)')

Missing value
*************************
Nombre de lignes : 935698
IDENTITY_ATTACK       :           0 (0%)
INSULT                :           0 (0%)
PROFANITY             :           0 (0%)
SEVERE_TOXICITY       :           0 (0%)
THREAT                :           0 (0%)
TOXICITY              :           0 (0%)
comment_count         :           0 (0%)
created_time_comments :           0 (0%)
id_comments           :           0 (0%)
like_count            :           0 (0%)
message_comments      :           0 (0%)
parent                :      459661 (49%)
postId                :           0 (0%)
attachments.data      :          35 (0%)
created_time_posts    :           0 (0%)
id_posts              :           0 (0%)
mainTopic             :        8481 (1%)
message_posts         :       10158 (1%)
permalink_url         :           0 (0%)
secondTopic           :       67076 (7%)
shares                :           0 (0%)
title                 :        8457 (1%)


***
## Variable de temps
***

In [151]:
# ====== convertir le temps en format datetime ====== #
df_merge['comment_time'] = pd.to_datetime(df_merge['created_time_comments'])
df_merge['post_time'] = pd.to_datetime(df_merge['created_time_posts'])
df_merge = df_merge.drop(columns=['created_time_comments', 'created_time_posts'])
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,id_comments,like_count,message_comments,parent,postId,attachments.data,id_posts,mainTopic,message_posts,permalink_url,secondTopic,shares,title,comment_time,post_time
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,722477749883613_788168749338745,0,"Faux, ce ne sera jamais le temps pour lui car ...",NaN,192978590727638_722477749883613,"[{""title"":""Les incohérences de l’ingérence"",""u...",192978590727638_722477749883613,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,https://www.facebook.com/553956616735728/posts...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,2023-06-02 16:06:59+00:00,2023-06-02 16:03:24+00:00
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,722477749883613_1593974021087773,1,Mon dieu!! On nous prend sérieusement pour des...,NaN,192978590727638_722477749883613,"[{""title"":""Les incohérences de l’ingérence"",""u...",192978590727638_722477749883613,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,https://www.facebook.com/553956616735728/posts...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,2023-06-02 16:21:43+00:00,2023-06-02 16:03:24+00:00


In [152]:
# ====== Éliminer les colonnes non-pertinantes ====== #
colones_non_utiles = ['id_comments', 'parent', 'postId', 'attachments.data', 'id_posts', 'permalink_url']
df_merge = df_merge.drop(columns=colones_non_utiles)
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,comment_time,post_time
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,2023-06-02 16:06:59+00:00,2023-06-02 16:03:24+00:00
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,2023-06-02 16:21:43+00:00,2023-06-02 16:03:24+00:00


In [153]:
# Attribut différence de temps
df_merge['time_difference'] = df_merge['comment_time'] - df_merge['post_time'] 
df_merge['time_difference'] = df_merge['time_difference'].astype('timedelta64[m]')
df_merge.head(2)

# Attributs temps solo
df_merge['year'] = df_merge['comment_time'].dt.year
df_merge['month'] = df_merge['comment_time'].dt.month
df_merge['weekday'] = df_merge['comment_time'].dt.weekday       # Monday=0, Sunday=6
df_merge = df_merge.drop(columns=['comment_time', 'post_time'])
df_merge.head(2)


,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4


In [154]:
# Attention au Nan
df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')

C:\Users\Charles_tour\AppData\Local\Temp\ipykernel_22784\284039997.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['message_comments', 'mainTopic', 'message_posts', 'secondTopic',
       'title'],
      dtype='object')]

  df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')


***
## Les sujets : Étiquettage
***

In [27]:
df_merge[["title"]]

,title
0,Les incohérences de l’ingérence
1,Legault veut empêcher les antiavortement de fa...
2,S’envoyer en l’air à 97 ans
3,Une famille québécoise vit dans une géonef aut...
4,Influenceurs : gare aux publicités trompeuses
...,...
475371,La CAQ prévoit améliorer les conditions de tra...
475372,La CAQ prévoit améliorer les conditions de tra...
475373,La CAQ prévoit améliorer les conditions de tra...
475374,La CAQ prévoit améliorer les conditions de tra...


***
## Outliers
***

***** Note : Je ne suis pas certain qu'on veuille enlever les outliers, puisque c'est probablement ces observation (avec de grosses valeurs) qui recoivent des commentaires.
Peut-être mettre les valeurs > upperbound = upperbound

In [15]:
df_merge =pd.read_hdf(cwd + '\\dataframe\\' + 'df_merge.h5')
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4


In [35]:
def countNumberOutlier(df, col):
    nbval_initial = len(df[col])
    
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    nbval_final = len(df.loc[df[col]<upper_bound])
    perte = nbval_initial - nbval_final
    perte_proportion = round(perte/nbval_initial, 2)

    print('*'*25)
    print(f'Colonne : {col}')
    print(f'Nombre d\'observation initial : {nbval_initial}')
    print(f'Nombre d\'observation sans outliers : {nbval_final}')
    print(f'Perte : {perte} ({perte_proportion} %)')

In [36]:
cols = ['IDENTITY_ATTACK', 'INSULT', 'PROFANITY', 'SEVERE_TOXICITY', 'THREAT', 'TOXICITY', 'comment_count', 'like_count']
df = df_merge.copy()

for col in cols:
    countNumberOutlier(df, col)

*************************
Colonne : IDENTITY_ATTACK
Nombre d'observation initial : 935698
Nombre d'observation sans outliers : 807410
Perte : 128288 (0.14 %)
*************************
Colonne : INSULT
Nombre d'observation initial : 935698
Nombre d'observation sans outliers : 812232
Perte : 123466 (0.13 %)
*************************
Colonne : PROFANITY
Nombre d'observation initial : 935698
Nombre d'observation sans outliers : 770424
Perte : 165274 (0.18 %)
*************************
Colonne : SEVERE_TOXICITY
Nombre d'observation initial : 935698
Nombre d'observation sans outliers : 795753
Perte : 139945 (0.15 %)
*************************
Colonne : THREAT
Nombre d'observation initial : 935698
Nombre d'observation sans outliers : 819678
Perte : 116020 (0.12 %)
*************************
Colonne : TOXICITY
Nombre d'observation initial : 935698
Nombre d'observation sans outliers : 869007
Perte : 66691 (0.07 %)
*************************
Colonne : comment_count
Nombre d'observation initial : 935

In [ ]:
# Valeur > upperbound = upperbound
for col in cols:
    df[col] = np.where(test[df] > test[df].quantile(0.75), test[df].quantile(0.75), test[df])

In [ ]:
df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')

***
## Transformation
***

In [7]:
#====== Mesure de l'assymétrie (skewness Kurtosis) ======#

# Skewness
#       Skewness positif : La queue de la distribution est étendue du côté droit
#       Skewness négatif : La queue de la distribution est étendue du côté gauche 
#       Skewness nul : La distribution est symétrique par rapport à la moyenne
#               skewness est entre -0.5 et 0.5, la distribution est relativement symétrique.

# Kurtosis
#       Kurtosis élevé (positif) : Les queues de la distribution sont plus lourdes que la distribution normale, et les pics sont plus élevés et plus étroits. Cela indique des valeurs aberrantes plus importantes.
#       Kurtosis faible (négatif) : Les queues de la distribution sont plus légères que la distribution normale, et les pics sont plus plats. Cela indique des valeurs aberrantes moins importantes.
#       Kurtosis standard (zéro) : La distribution a la même forme que la distribution normale (mesurée en termes de pics et de queues).
#               Un kurtosis entre -2 et 2 est considéré comme normal (mesuré par rapport à une distribution normale).
#               Un kurtosis supérieur à 2 indique une distribution plus « lourde » que la normale (plus de valeurs dans les queues).

import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis

df_merge =pd.read_hdf(cwd + '\\dataframe\\' + 'df_merge.h5')
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4


In [ ]:
# cols = ['IDENTITY_ATTACK', 'INSULT', 'PROFANITY', 'SEVERE_TOXICITY', 'THREAT', 'TOXICITY', 'comment_count', 'like_count']
# df = df_merge[cols].copy()

# for col in cols: 
#     df[col+'_log'] = np.log1p(df[col])
#     df[col+'_cube'] = np.power(df[col], 1/3)
# df.head(2)

In [8]:
cols = ['IDENTITY_ATTACK', 'INSULT', 'PROFANITY', 'SEVERE_TOXICITY', 'THREAT', 'TOXICITY', 'comment_count', 'like_count', 'shares']
for col in cols: 
    # df_merge[col+'_log'] = np.log1p(df_merge[col])
    df_merge[col+'_cube'] = np.power(df_merge[col], 1/3)
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,IDENTITY_ATTACK_cube,INSULT_cube,PROFANITY_cube,SEVERE_TOXICITY_cube,THREAT_cube,TOXICITY_cube,comment_count_cube,like_count_cube,shares_cube
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,0.467562,0.866814,0.872661,0.705121,0.310492,0.838017,0.0,0.0,0.0
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,0.176257,0.551253,0.634546,0.191119,0.178081,0.560745,1.0,1.0,0.0


In [9]:
df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')

C:\Users\Charles_tour\AppData\Local\Temp\ipykernel_20500\1801815014.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['message_comments', 'mainTopic', 'message_posts', 'secondTopic',
       'title'],
      dtype='object')]

  df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')


In [157]:
def printSkewAndKurtosis(df, col):
    print('*'*25)
    print(f'Colonne : {col}')
    print('moyenne : ', df[col].mean())
    print('mediane : ', df[col].median())
    print('skewness : ', round(skew(df[col])))
    print('Kurtosis :', round(kurtosis(df[col])))

def printHistogram(df, col):
    plt.hist(df[col])
    plt.title('comment_count')
    plt.ylabel('Fréquence')
    plt.xlabel('Value')
    plt.show()

def generateStatsValues(df, col):
    moyenne = round(df[col].mean(), 2)
    mediane = round(df[col].median(), 2)
    skew_value = round(skew(df[col]), 2)
    kurtosis_value = round(kurtosis(df[col]), 2)
    return moyenne, mediane, skew_value, kurtosis_value

In [158]:
stats_dict = {}

cols_to_dict = df_merge.columns.to_list() 

for col in cols_to_dict:
    stats_dict[col] = generateStatsValues(df_merge, col)

stats_df = pd.DataFrame(stats_dict).transpose().rename(columns={0:'Moyenne', 1:'Mediane', 2:'Skewness', 3:'Kurtosis'}).sort_index()
stats_df

# Rappel : 
# Skewness est entre -0.5 et 0.5
# Kurtosis entre -2 et 2 est considéré comme normal

***
## Extraction des mots dans les publications
***

1. Mots pertinent (les titres et les commentaires, les commentaires parents)
2. Les émoji

In [57]:
df_merge =pd.read_hdf(cwd + '\\dataframe\\' + 'df_merge.h5')
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,IDENTITY_ATTACK_cube,INSULT_cube,PROFANITY_cube,SEVERE_TOXICITY_cube,THREAT_cube,TOXICITY_cube,comment_count_cube,like_count_cube,shares_cube,messages_clean,emojis,emojis_text,positive,neutral,negative
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,0.467562,0.866814,0.872661,0.705121,0.310492,0.838017,0.0,0.0,0.0,faux jamais temps réélu parti crève faim jamai...,[],[],0.077610,0.130838,0.791552
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,0.176257,0.551253,0.634546,0.191119,0.178081,0.560745,1.0,1.0,0.0,dieu prend sérieusement idiots 😅,[😅],[grinning_face_with_sweat],0.697028,0.047872,0.255100


In [79]:
df_test = df_merge.head(2000).copy()

In [80]:
# Nettoyer stop words avec spacy
import spacy
import concurrent.futures

import re

nlp = spacy.load('fr_core_news_sm')

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)


def cleanTextWithSpacy(text):
    text = remove_emojis(text)
    doc = nlp(text)
    clean_tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
    clean_text = ' '.join(clean_tokens)
    clean_text = clean_text.replace('\n', ' ').replace('  ', ' ').strip().replace('¿', '').replace('¡', '')
    clean_text = clean_text.replace(u'«', '').replace(u'»', '').replace("'", '').replace("-", ' ')
    return clean_text


def applyCleaningText(df, initial_col_name, final_col_name):
    df[final_col_name] = df[initial_col_name].apply(lambda x: cleanTextWithSpacy(x))
    return df


# with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
#     results = list(executor.map(applyCleaningText(df_test, 'message_comments', 'messages_clean')))

df_test = applyCleaningText(df_test, 'message_comments', 'messages_clean')
df_test.head(5)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,IDENTITY_ATTACK_cube,INSULT_cube,PROFANITY_cube,SEVERE_TOXICITY_cube,THREAT_cube,TOXICITY_cube,comment_count_cube,like_count_cube,shares_cube,messages_clean,emojis,emojis_text,positive,neutral,negative
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,0.467562,0.866814,0.872661,0.705121,0.310492,0.838017,0.0,0.0,0.0,Faux jamais temps réélu parti crève faim Jamai...,[],[],0.077610,0.130838,0.791552
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,0.176257,0.551253,0.634546,0.191119,0.178081,0.560745,1.0,1.0,0.0,dieu prend sérieusement idiots,[😅],[grinning_face_with_sweat],0.697028,0.047872,0.255100
2,0.029144,0.269578,0.016753,0.006104,0.005864,0.246033,0,0,"Denise Morais ..Oui et avec raison ! (Car ""nou...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,86.0,2023,6,4,0.307741,0.645994,0.255876,0.182751,0.180331,0.626611,0.0,0.0,0.0,Denise Morais Oui raison majorité tyrannique ...,[],[],0.243069,0.089332,0.667599
3,0.036105,0.402434,0.479813,0.023411,0.008220,0.372227,0,0,Gros show. Mais pendant la plandémie chinoise ...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,94.0,2023,6,4,0.330514,0.738298,0.782872,0.286072,0.201820,0.719343,0.0,0.0,0.0,Gros show plandémie chinoise tlm fermait bien ...,[],[],0.685404,0.125043,0.189553
4,0.280349,0.520819,0.163573,0.169603,0.008790,0.439655,0,0,On apellait ça chinada pis les vaxxinés compre...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,95.0,2023,6,4,0.654485,0.804567,0.546895,0.553534,0.206378,0.760391,0.0,0.0,0.0,apellait chinada pis vaxxinés comprenaient rie...,[],[],0.158446,0.242306,0.599248


In [82]:
# Apyori test
import pandas as pd
from collections import Counter
import nltk
from apyori import apriori

# Tokenize text into words
def tokenize_text(text):
    tokens = nltk.word_tokenize(text)
    return tokens

# Preprocess the text data
message_data = df_test['messages_clean'].tolist()
tokenized_messages = [tokenize_text(message) for message in message_data]
message_sets = [set(tokens) for tokens in tokenized_messages]

# Run Apriori algorithm
results = list(apriori(message_sets, min_support=0.05, min_confidence=0.1))
results

[]

In [61]:
df_test[['message_comments', 'messages_clean']].to_excel('message.xlsx')

In [11]:
# Nettoyer stop words avec spacy
# import spacy
# nlp = spacy.load('fr_core_news_sm')

# def cleanTextWithSpacy(text):
#     doc = nlp(text)
#     filtered_words = [token.text for token in doc if (not token.is_stop) & (not token.is_punct)]
#     clean_text = ' '.join(filtered_words).lower()
#     return clean_text

# df_merge['messages_clean'] = df_merge['message_comments'].apply(lambda x: cleanTextWithSpacy(x))
# df_merge.head()

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,IDENTITY_ATTACK_cube,INSULT_cube,PROFANITY_cube,SEVERE_TOXICITY_cube,THREAT_cube,TOXICITY_cube,comment_count_cube,like_count_cube,shares_cube,messages_clean
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,0.467562,0.866814,0.872661,0.705121,0.310492,0.838017,0.0,0.0,0.0,faux jamais temps réélu parti crève faim jamai...
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,0.176257,0.551253,0.634546,0.191119,0.178081,0.560745,1.0,1.0,0.0,dieu prend sérieusement idiots 😅
2,0.029144,0.269578,0.016753,0.006104,0.005864,0.246033,0,0,"Denise Morais ..Oui et avec raison ! (Car ""nou...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,86.0,2023,6,4,0.307741,0.645994,0.255876,0.182751,0.180331,0.626611,0.0,0.0,0.0,denise morais oui raison majorité tyrannique...
3,0.036105,0.402434,0.479813,0.023411,0.008220,0.372227,0,0,Gros show. Mais pendant la plandémie chinoise ...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,94.0,2023,6,4,0.330514,0.738298,0.782872,0.286072,0.201820,0.719343,0.0,0.0,0.0,gros show plandémie chinoise tlm fermait bien ...
4,0.280349,0.520819,0.163573,0.169603,0.008790,0.439655,0,0,On apellait ça chinada pis les vaxxinés compre...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,95.0,2023,6,4,0.654485,0.804567,0.546895,0.553534,0.206378,0.760391,0.0,0.0,0.0,apellait chinada pis vaxxinés comprenaient rie...


In [12]:
# Extraire les Émoji
import re
import emoji

def findEmoji(text):
    emojis = re.findall(r'[\U0001F300-\U0001F5FF|\U0001F600-\U0001F64F|\U0001F680-\U0001F6FF|\u2600-\u26FF\u2700-\u27BF]', text)
    return emojis

##### Créer une liste de valeur plutot qu'un bloc de text #####
def emojiToText(text):
    emojiInText = emoji.demojize(text)
    return emojiInText

df_merge['emojis'] = df_merge['messages_clean'].apply(lambda x: findEmoji(x))
df_merge['emojis_text'] = df_merge['emojis'].apply(lambda x: emojiToText(x))
df_merge['emojis_text'] = df_merge['emojis_text'][:30].str[1:-1].apply(lambda x: x.split('::'))

df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,IDENTITY_ATTACK_cube,INSULT_cube,PROFANITY_cube,SEVERE_TOXICITY_cube,THREAT_cube,TOXICITY_cube,comment_count_cube,like_count_cube,shares_cube,messages_clean,emojis,emojis_text
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,0.467562,0.866814,0.872661,0.705121,0.310492,0.838017,0.0,0.0,0.0,faux jamais temps réélu parti crève faim jamai...,[],[]
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,0.176257,0.551253,0.634546,0.191119,0.178081,0.560745,1.0,1.0,0.0,dieu prend sérieusement idiots 😅,[😅],[grinning_face_with_sweat]


In [13]:
df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')

C:\Users\Charles_tour\AppData\Local\Temp\ipykernel_20500\1801815014.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['message_comments', 'mainTopic', 'message_posts', 'secondTopic',
       'title', 'messages_clean', 'emojis', 'emojis_text'],
      dtype='object')]

  df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')


***
## Topics
***

In [83]:
df_merge = pd.read_hdf(cwd + '\\dataframe\\' + 'df_merge.h5')
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,IDENTITY_ATTACK_cube,INSULT_cube,PROFANITY_cube,SEVERE_TOXICITY_cube,THREAT_cube,TOXICITY_cube,comment_count_cube,like_count_cube,shares_cube,messages_clean,emojis,emojis_text,positive,neutral,negative
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,0.467562,0.866814,0.872661,0.705121,0.310492,0.838017,0.0,0.0,0.0,faux jamais temps réélu parti crève faim jamai...,[],[],0.077610,0.130838,0.791552
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,0.176257,0.551253,0.634546,0.191119,0.178081,0.560745,1.0,1.0,0.0,dieu prend sérieusement idiots 😅,[😅],[grinning_face_with_sweat],0.697028,0.047872,0.255100


In [97]:
df_topics = df_merge[['secondTopic']].copy()

df_topics = df_topics.dropna().drop_duplicates() 
df_topics = df_topics.loc[(df_topics['secondTopic'].str.len()<15) & (df_topics['secondTopic'].str.len()>4)]

df_topics.to_excel('secondTopics.xlsx')
df_topics

,secondTopic
75,dans-le-verre
86,hockey-junior
150,point-de-vue
163,tele-et-radio
379,baseball
...,...
773820,histoire
790333,energir
790334,la-sepaq
884548,thetford


In [132]:
df_test = df_merge.copy()

topics_dicts = {
    'sport' :       ['sport', 'hockey', 'baseball', 'football', 'canadien', 'remparts', 'rouge-et-or', 'olympiques'], 
    'musique' :     ['musique'],
    'sante' :       ['sante'],
    'politique' :   ['politique', 'senateurs'],
    'education' :   ['education'],
    'cinema' :      ['cinema'],
    'arts' :        ['arts'],
    'celebrites' :  ['celebrites'],
    'Municipalité' : ['Municipalité', 'capitale', 'gatineau', 'sherbrooke', 'ottawa', 'trois-rivieres', 'thetford'],
    'voyage' :      ['voyage' ],
    'science':      ['science'],
    'environnement' : ['environnement'],
    'covid' :       ['covid'],
    'techno' :      ['techno'],
    'histoire' :    ['histoire']
}

def check_topic(x):
    for key, value in topics_dicts.items():
        if x in value:
            return key
    return None

df_test['topic'] = df_test['secondTopic'].apply(check_topic)

# df_test['topic'] = np.where((df_test['secondTopic'].isin(topics_list) | (df_test['secondTopic'].isin(flattened_topics))), df_test['secondTopic'], None)


df_test

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,IDENTITY_ATTACK_cube,INSULT_cube,PROFANITY_cube,SEVERE_TOXICITY_cube,THREAT_cube,TOXICITY_cube,comment_count_cube,like_count_cube,shares_cube,messages_clean,emojis,emojis_text,positive,neutral,negative,topic
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,0.467562,0.866814,0.872661,0.705121,0.310492,0.838017,0.0,0.0,0.000000,faux jamais temps réélu parti crève faim jamai...,[],[],0.077610,0.130838,0.791552,None
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,0.176257,0.551253,0.634546,0.191119,0.178081,0.560745,1.0,1.0,0.000000,dieu prend sérieusement idiots 😅,[😅],[grinning_face_with_sweat],0.697028,0.047872,0.255100,None
2,0.029144,0.269578,0.016753,0.006104,0.005864,0.246033,0,0,"Denise Morais ..Oui et avec raison ! (Car ""nou...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,86.0,2023,6,4,0.307741,0.645994,0.255876,0.182751,0.180331,0.626611,0.0,0.0,0.000000,denise morais oui raison majorité tyrannique...,[],[],0.243069,0.089332,0.667599,None
3,0.036105,0.402434,0.479813,0.023411,0.008220,0.372227,0,0,Gros show. Mais pendant la plandémie chinoise ...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,94.0,2023,6,4,0.330514,0.738298,0.782872,0.286072,0.201820,0.719343,0.0,0.0,0.000000,gros show plandémie chinoise tlm fermait bien ...,[],[],0.685404,0.125043,0.189553,None
4,0.280349,0.520819,0.163573,0.169603,0.008790,0.439655,0,0,On apellait ça chinada pis les vaxxinés compre...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,95.0,2023,6,4,0.654485,0.804567,0.546895,0.553534,0.206378,0.760391,0.0,0.0,0.000000,apellait chinada pis vaxxinés comprenaient rie...,[],[],0.158446,0.242306,0.599248,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935693,0.003515,0.128838,0.021467,0.003014,0.005594,0.133882,0,0,Marcel Routhier si tu n'as pas entendu parler ...,actualite,« Il y a une chose que les infirmières veulent...,politique,7,La CAQ prévoit améliorer les conditions de tra...,2069.0,2021,9,3,0.152043,0.505065,0.277922,0.144443,0.177517,0.511573,0.0,0.0,1.912931,marcel routhier entendu baisse taxe scolaire e...,[],NaN,0.066566,0.255113,0.678320,politique
935694,0.000370,0.008805,0.008153,0.000207,0.005254,0.005246,0,0,"Gaston Crete en 2002 c,était Rémy Trudel et 20...",actualite,« Il y a une chose que les infirmières veulent...,politique,7,La CAQ prévoit améliorer les conditions de tra...,2519.0,2021,9,3,0.071789,0.206494,0.201269,0.059195,0.173843,0.173761,0.0,0.0,1.912931,gaston crete 2002 c rémy trudel 2003 françois ...,[],NaN,0.451840,0.213279,0.334880,politique
935695,0.005217,0.130648,0.019007,0.002956,0.005476,0.119129,0,0,"Gaston Crete et c,est là Marois qui a parti le...",actualite,« Il y a une chose que les infirmières veulent...,politique,7,La CAQ prévoit améliorer les conditions de tra...,2522.0,2021,9,3,0.173433,0.507420,0.266874,0.143523,0.176259,0.492047,0.0,0.0,1.912931,gaston crete c marois parti bal donnant belle ...,[],NaN,0.801224,0.069705,0.129071,politique
935696,0.012061,0.125217,0.014942,0.004997,0.005926,0.115658,0,0,Ils commencent à se fâcher le monde ne veulen...,actualite,« Il y a une 

In [137]:
list(topics_dicts.keys())

['sport',
 'musique',
 'sante',
 'politique',
 'education',
 'cinema',
 'arts',
 'celebrites',
 'Municipalité',
 'voyage',
 'science',
 'environnement',
 'covid',
 'techno',
 'histoire']

In [136]:
df_model = df_test[['message_posts', 'topic']].copy().drop_duplicates(subset='message_posts').reset_index(drop=True)

df_model.loc[df_model['topic'].notnull()]

,message_posts,topic
33,«Le frappeur a atteint le gant du receveur ave...,sport
36,«Quand un jeune se fait crier après suite à un...,sport
37,"Pour le nouvel entraîneur-chef, la rétrogradat...",sport
42,Les gens de la région de Québec célébreront en...,musique
43,Le Canada devient le premier pays au monde à a...,sante
...,...,...
25178,"Pendant des décennies, les vaccins à ARN avaie...",science
25207,Ses problème ont culminé avant le match du 23 ...,sport
25230,François Legault était intervenu dans la campa...,politique
25234,"Si c’était à refaire, François Legault assure ...",politique


In [141]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# List of possible topics
topic_list = ['sport', 'musique', 'sante', 'politique', 'education', 'cinema', 'arts', 'celebrites', 'Municipalité', 'voyage', 'science', 'environnement', 'covid', 'techno', 'histoire']


# Split data into train and test sets
train_data = df_model[~df_model['message_posts'].isna()]
test_data = df_model[df_model['message_posts'].isna()]

train_data['topic'] = train_data['topic'].fillna('')
test_data['topic'] = test_data['topic'].fillna('')

# Vectorize text data
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_data['topic'])
X_test = vectorizer.transform(test_data['topic'])

# Train a Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, train_data['message_posts'])

# Predict topics for missing values
predicted_topics = clf.predict(X_test)

# Convert NumPy strings to Python strings
predicted_topics = [str(topic) for topic in predicted_topics]

# Update the DataFrame with predicted topics
test_data['Predicted_Topic'] = [topic_list[topic_list.index(topic)] for topic in predicted_topics]

# Concatenate train and test data
df_updated = pd.concat([train_data, test_data])

df_updated

C:\Users\Charles_tour\AppData\Local\Temp\ipykernel_20500\280344599.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['topic'] = train_data['topic'].fillna('')
C:\Users\Charles_tour\AppData\Local\Temp\ipykernel_20500\280344599.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['topic'] = test_data['topic'].fillna('')


ValueError: '(CONTENU COMMANDITÉ) - Le malheur s’abat sur vous par la main de la maladie et vous voilà au départ de votre parcours dans le réseau de la santé. Le contact, l’empathie, l’échange d’émotions comptent pour beaucoup dans les soins. Mais pour assurer le suivi de votre état, il est nécessaire de collecter des données lesquelles vous seront, au final, livrées par une personne compétente pour les bonifier de ses explications.' is not in list

In [84]:
df_test = df_merge.copy().head(100)
# message_list = df_test['messages_clean'].tolist()
# message_list

In [35]:
# Objectif : isoler les itemsets fréquent dont un commentaire est présent
from apyori import apriori

df_test = df_merge.copy().head(1000)
message_list = df_test['messages_clean'].tolist()


results = list(apriori(message_list,  min_support=0.50,  min_confidence=1))
results

[RelationRecord(items=frozenset({'p', 'e', 'c'}), support=0.516, ordered_statistics=[OrderedStatistic(items_base=frozenset({'p', 'c'}), items_add=frozenset({'e'}), confidence=1.0, lift=1.0582010582010584)]),
 RelationRecord(items=frozenset({'p', 'e', ' ', 'c'}), support=0.514, ordered_statistics=[OrderedStatistic(items_base=frozenset({'p', ' ', 'c'}), items_add=frozenset({'e'}), confidence=1.0, lift=1.0582010582010584)]),
 RelationRecord(items=frozenset({'a', 'p', 'e', 'c'}), support=0.502, ordered_statistics=[OrderedStatistic(items_base=frozenset({'a', 'p', 'c'}), items_add=frozenset({'e'}), confidence=1.0, lift=1.0582010582010584)]),
 RelationRecord(items=frozenset({'p', 'r', 'e', 'c'}), support=0.503, ordered_statistics=[OrderedStatistic(items_base=frozenset({'p', 'r', 'c'}), items_add=frozenset({'e'}), confidence=1.0, lift=1.0582010582010584)]),
 RelationRecord(items=frozenset({'p', ' ', 'c', 'a', 'e'}), support=0.5, ordered_statistics=[OrderedStatistic(items_base=frozenset({'a', '

In [31]:
message_list

['faux jamais temps réélu parti crève faim jamais prêts campagne électorale coffres désespérément vides unique raison transformé lèche cul parti libéral',
 'dieu prend sérieusement idiots 😅',
 'denise morais oui raison   majorité tyrannique citoyens vendus   dociles      servitude volontaire exceptions   expérimenté solide   mckinsey incongruences politiciens mentez mentez restera chose québec gagnant formatés solide',
 'gros show plandémie chinoise tlm fermait bien gueule',
 'apellait chinada pis vaxxinés comprenaient rien vois ben yont cerveau',
 'chinadaaaaaa',
 'trop drôle   crevé caves allez voir cyclistes leggings   payant',
 'construit copie marché champlain bonbon électoral bientôt oubliette$ 🤞 🕳 caq',
 'québec chantier été',
 "temps amusez -vous queers amis belzebuth discernement été distribué égal   spectales legault   \n bass drum of death dearly beloved basement revolver anti bar spectacles québec qc canada \n jeudi 08 juin 2023 \n stick to your guns don' t try \n louis-jea

In [151]:
# Extraire les sujet avec un commentaire
print(len(df_merge))
df = df_merge.loc[(df_merge['comment_count']>0)].copy().dropna(subset='mainTopic')
print(len(df))

len(df['mainTopic'].unique())



935698
114979


901

In [156]:
test = df.loc[(df['mainTopic'].str.contains('Facebook') == False) & ((df['mainTopic'].str.len() < 20))]
# df['mainTopic'].str.len() > 10
print(len(test['mainTopic'].unique()))
test['mainTopic'].unique()
# test['mainTopic']


45


array(['composer-preview', 'actualites', 'affaires', 'arts', 'monde',
       'sports', 'vivre', 'chroniques', 'opinions', 'consommation',
       'partenaires', '553956616735728', 'jeunesse', '2023', 'magazines',
       'lesoleildequebec', '2022', 'actualite', 'le-mag', 'la-vitrine',
       'science', '192978590727638', 'cinema', 'zone', 'auto', 'maison',
       'covid-19', 'dossiers', 'programmation', 'quebec-billets',
       'hqaRGaI', 'toit-et-moi', 'hockey-junior', 'nos-recommandations',
       'remparts', '34DqsI9', 'ma-region', 'events', '2Ncrl4z',
       'protegez-vous', 'je-mabonne', 'articles', 'tokyo-2021',
       'lesasdelinfo', 'claude-villeneuve'], dtype=object)

In [166]:
test2 = test.drop_duplicates(subset='title').sort_values(by=["comment_count"], ascending=False).reset_index(drop=True)
print(len(test2))
test2.head(5)

14414


,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday
0,0.008251,0.473846,0.114161,0.018579,0.007482,0.360951,194,11,"Pauvre MICKAËL BERGERON,\nC’est pas facile d’ê...",ma-region,Chronique / On savait déjà que des gens allaie...,la-theorie-du-complot-rattrapee-par-la-realite...,124,La théorie du complot rattrapée par la réalité,42.0,2020,12,1
1,0.002867,0.159116,0.012825,0.002394,0.005448,0.157115,178,19,Ça serait bien que vous compreniez que nous av...,actualite,Les policiers de la Sûreté du Québec (SQ) sont...,en-region,25,COVID-19: violence et débordements dans des ba...,11.0,2020,12,0
2,0.007696,0.027476,0.008623,0.003109,0.005541,0.037106,145,60,8 enfants avec les allocations familiales ou e...,actualites,De purs inconnus ont laissé une trace indélébi...,des-anges-gardiens-offrent-un-nouveau-depart-a...,725,Des « anges gardiens » offrent un nouveau dépa...,137.0,2020,12,6
3,0.009027,0.155564,0.011475,0.003033,0.005437,0.130411,109,0,Pourquoi l'OMS se tourne vers un chercheur d'u...,actualites,L’Organisation mondiale de la santé (OMS) a de...,un-sherbrookois-recommande-a-loms-le-recours-a...,7,Un Sherbrookois recommande à l'OMS le recours ...,10.0,2020,9,4
4,0.001628,0.021353,0.009212,0.001936,0.005670,0.027560,109,10,7 sur 10 refuse cette thérapie génique. Très ...,actualites,Un total de 4400 travailleurs du milieu de la ...,covid-19,5,Les employés des CHSLD vaccinés en priorité en...,3.0,2020,12,0


In [169]:
test2['mainTopic'].unique()
test2['secondTopic'].unique()

array(['la-theorie-du-complot-rattrapee-par-la-realite-1b2ab9f05009fce9ffad99f5f931e0b1?utm_source=dlvr.it&utm_medium=facebook',
       'en-region',
       'des-anges-gardiens-offrent-un-nouveau-depart-a-une-famille-dans-le-besoin-87ae5f034b0418f45a7084b4e922fa9d?utm_source=dlvr.it&utm_medium=facebook',
       ...,
       'covid-19-francois-bonnardel-place-en-isolement-11d7c820b673493bda33dae60f268132?utm_source=dlvr.it&utm_medium=facebook',
       'la-relation-malsaine-entre-donald-trump-et-qanon-6533a9fdecd24d10d3954040fa3dc8e3?utm_source=dlvr.it&utm_medium=facebook',
       'offre-dachat-de-cogeco-la-famille-audet-reitere-sa-fin-de-non-recevoir-672f277d36562ee55858403102b30d11?utm_source=dlvr.it&utm_medium=facebook'],
      dtype=object)

***
## Analyse de sentiments
***

In [17]:
df_merge = pd.read_hdf(cwd + '\\dataframe\\' + 'df_merge.h5')
df_multi_student = pd.read_hdf(cwd + '\\dataframe\\' + 'sentiement_model_results.h5')

df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,IDENTITY_ATTACK_cube,INSULT_cube,PROFANITY_cube,SEVERE_TOXICITY_cube,THREAT_cube,TOXICITY_cube,comment_count_cube,like_count_cube,shares_cube,messages_clean,emojis,emojis_text
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,0.467562,0.866814,0.872661,0.705121,0.310492,0.838017,0.0,0.0,0.0,faux jamais temps réélu parti crève faim jamai...,[],[]
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,0.176257,0.551253,0.634546,0.191119,0.178081,0.560745,1.0,1.0,0.0,dieu prend sérieusement idiots 😅,[😅],[grinning_face_with_sweat]


In [137]:
from transformers import pipeline
df = df_merge[['message_comments']].copy()
list_to_process = df['message_comments'].tolist()

In [139]:
# Model
distilled_student_sentiment_classifier = pipeline(model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", return_all_scores=True, truncation=True)
result_multi_student = distilled_student_sentiment_classifier(list_to_process)

# Change to df
data = [[d["score"] for d in result_multi_student[i]] for i in range(len(result_multi_student))]
col_names = ['positive', 'neutral', 'negative']
df_multi_student = pd.DataFrame(data, columns=col_names)

In [140]:
# Save results
df_multi_student.to_hdf(cwd + '\\dataframe\\' + 'sentiement_model_results.h5', key='s')

In [124]:
# Model
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
sentiment_pipeline_multilingual = pipeline('sentiment-analysis', model=model_name, tokenizer=model_name)
result_multi = sentiment_pipeline_multilingual(list_to_process, return_all_scores=True)

# Change to df
data = [[d["score"] for d in result_multi[i]] for i in range(len(result_multi))]
col_names = ['1stars', '2stars', '3stars', '4stars', '5stars']
df_multi = pd.DataFrame(data, columns=col_names)

In [141]:
# df_test = pd.concat([df, df_multi, df_multi_student], axis='columns')
# df_test = pd.concat([df, df_multi_student], axis='columns')

# df_test

,message_comments,positive,neutral,negative
0,"Faux, ce ne sera jamais le temps pour lui car ...",0.077610,0.130838,0.791552
1,Mon dieu!! On nous prend sérieusement pour des...,0.697028,0.047872,0.255100
2,"Denise Morais ..Oui et avec raison ! (Car ""nou...",0.243069,0.089332,0.667599
3,Gros show. Mais pendant la plandémie chinoise ...,0.685404,0.125043,0.189553
4,On apellait ça chinada pis les vaxxinés compre...,0.158446,0.242306,0.599248
...,...,...,...,...
935693,Marcel Routhier si tu n'as pas entendu parler ...,0.066566,0.255113,0.678320
935694,"Gaston Crete en 2002 c,était Rémy Trudel et 20...",0.451840,0.213279,0.334880
935695,"Gaston Crete et c,est là Marois qui a parti le...",0.801224,0.069705,0.129071
935696,Ils commencent à se fâcher le monde ne veulen...,0.187257,0.304669,0.508074


In [20]:
df_merge = pd.concat([df_merge, df_multi_student], axis='columns')
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,IDENTITY_ATTACK_cube,INSULT_cube,PROFANITY_cube,SEVERE_TOXICITY_cube,THREAT_cube,TOXICITY_cube,comment_count_cube,like_count_cube,shares_cube,messages_clean,emojis,emojis_text,positive,neutral,negative
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,0.467562,0.866814,0.872661,0.705121,0.310492,0.838017,0.0,0.0,0.0,faux jamais temps réélu parti crève faim jamai...,[],[],0.077610,0.130838,0.791552
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,0.176257,0.551253,0.634546,0.191119,0.178081,0.560745,1.0,1.0,0.0,dieu prend sérieusement idiots 😅,[😅],[grinning_face_with_sweat],0.697028,0.047872,0.255100


In [21]:
df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')

C:\Users\Charles_tour\AppData\Local\Temp\ipykernel_20500\1801815014.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['message_comments', 'mainTopic', 'message_posts', 'secondTopic',
       'title', 'messages_clean', 'emojis', 'emojis_text'],
      dtype='object')]

  df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')
